In [21]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv("maindata.csv")#,header=1, engine='openpyxl')

In [23]:
df_for_classifier = df[['HoarseVoice',
       'DiffSwal', 'PainSwal', 'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump',
       'FOSIT', 'SoreThroat',"Diagnostic group"]]
df_for_classifier.head()

,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Non-cancer
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Non-cancer
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Non-cancer
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Non-cancer
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Non-cancer


In [25]:
#df_for_classifier['Gender'] = df_for_classifier['Gender'].map({'M': 1, 'F': 0})
df_for_classifier['Diagnostic group'] = df_for_classifier['Diagnostic group'].map({'Cancer': 1, 'Non-cancer': 0})
df_for_classifier.head()

,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
#df_for_classifier['Age'] = scaler.fit_transform(df_for_classifier[['Age']])
df_for_classifier['Ht'] = scaler.fit_transform(df_for_classifier[['Ht']])
df_for_classifier['Weight'] = scaler.fit_transform(df_for_classifier[['Weight']])
df_for_classifier.head()


/tmp/ipykernel_1860/3440211753.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Ht'] = scaler.fit_transform(df_for_classifier[['Ht']])
/tmp/ipykernel_1860/3440211753.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Weight'] = scaler.fit_transform(df_for_classifier[['Weight']])


,Gender,Ht,Weight,BMI,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,1,0.035088,0.000000,14.561642,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0.491228,0.029960,18.824626,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,0.684211,0.040773,19.094811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.842105,0.049043,19.302811,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.543860,0.035430,19.653979,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_for_classifier.columns

Index(['HoarseVoice', 'DiffSwal', 'PainSwal', 'Otalgia', 'UnilatNO',
       'NeckLump', 'ThroatLump', 'FOSIT', 'SoreThroat', 'Diagnostic group'],
      dtype='object')

In [31]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
pd.options.mode.chained_assignment = None
# Fill NaNs with 0
df_for_classifier.fillna(0, inplace=True)

# Define features and target variable
X = df_for_classifier[[ 'HoarseVoice', 'DiffSwal',
       'PainSwal', 'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump', 'FOSIT',
       'SoreThroat']]
y = df_for_classifier['Diagnostic group']

# Initialize StratifiedKFold
log_reg = LogisticRegression(class_weight={0:1,1:400}, max_iter=10000)
skf = StratifiedKFold(n_splits=10)

# List to store confusion matrices for each fold
conf_matrices = []


conf_matrices = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the model
    log_reg.fit(X_train, y_train)
    print(log_reg.coef_)    
    # Make predictions
    y_pred = log_reg.predict(X_test)
    
    # Compute the confusion matrix for this fold
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Normalize the confusion matrix so that all elements sum to 100%
    conf_matrix_normalized = conf_matrix / conf_matrix.sum() * 100
    conf_matrices.append(conf_matrix_normalized)
    print()
# Convert list of confusion matrices to a numpy array
conf_matrices = np.array(conf_matrices)

# Calculate the mean confusion matrix
mean_conf_matrix = np.mean(conf_matrices, axis=0)

# Calculate the standard deviation of the confusion matrices
std_dev_conf_matrix = np.std(conf_matrices, axis=0)

# Round the mean and standard deviation confusion matrices to 1 decimal place
mean_conf_matrix_rounded = np.round(mean_conf_matrix, 1)
std_dev_conf_matrix_rounded = np.round(std_dev_conf_matrix, 1)

# Print the rounded mean confusion matrix
print("Mean Confusion Matrix (Percentages, Rounded to 1 dp):")
print(mean_conf_matrix_rounded)

# Print the rounded standard deviation of the confusion matrices
print("Standard Deviation of Confusion Matrices (Percentages, Rounded to 1 dp):")
print(std_dev_conf_matrix_rounded)

# Optionally, print the classification report for the overall predictions
y_pred = cross_val_predict(log_reg, X, y, cv=10)
class_report = classification_report(y, y_pred)
print("Classification Report:")
print(class_report)

[[-1.39554087 -1.65830491  1.35542953  1.41172053 -1.54924475  0.81640876
   2.13088067 -2.3416003  -0.96031198]]

[[-1.06538192 -1.13833997  1.59956286  1.87403716 -1.25967876  0.14377419
   3.90058308 -4.50020232 -0.87167953]]

[[-1.38007935 -1.39884761  1.21089292  1.77609046 -1.47423851  0.51391519
   1.96936068 -1.99297421 -1.17641797]]

[[-1.75256539 -1.84237613  0.24119194  1.09969778 -1.84223691  0.22026077
   2.60857528 -1.9854337  -0.97953222]]

[[-1.30285896 -2.42573458  1.80248853  1.77584897 -1.3810845   0.42610133
   2.00818916 -1.85113945 -0.88763758]]

[[-1.56808572 -1.37728232  1.84198072  2.80512423 -6.47681646  2.53975861
   2.43894234 -1.29567625 -0.18733977]]

[[-1.42109972 -1.35694999  1.37943315  1.572487   -1.13565493  0.91639385
   2.19040812 -1.97001175 -1.28060139]]

[[-1.59171232 -1.3065045   1.78176873  1.58997291 -1.72655368  0.32324914
   1.33564449 -2.21243485 -1.37202809]]

[[-1.07232448 -2.17108682  0.10589559  4.60652459 -0.68804009  1.72867159
   0.5

In [18]:
log_reg.coef_

array([[ 3.36321554,  4.52980659, -0.25394122, -0.10116775, -3.06638628,
        -5.21110885,  0.75351795,  3.21647674, -4.46551128,  2.92649466,
         2.14261275, -1.14893185,  0.80583151]])

In [33]:
list(zip(['HoarseVoice', 'DiffSwal',
       'PainSwal', 'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump', 'FOSIT',
       'SoreThroat'],*log_reg.coef_.tolist()))

[('HoarseVoice', -1.3803580376288809),
 ('DiffSwal', -1.7112139809521427),
 ('PainSwal', 1.510088089870463),
 ('Otalgia', 1.349394787708064),
 ('UnilatNO', -1.3999667271339171),
 ('NeckLump', 0.9270017304315982),
 ('ThroatLump', 2.0931099792406154),
 ('FOSIT', -2.2893187106407855),
 ('SoreThroat', -0.8631519677293464)]